# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [661]:
# Importing modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express  as px


In [662]:
DEBUG=True

In [663]:
df = pd.read_csv('data/vehicles.csv')
#print(df['model'].unique())
#print(df[df['manufacturer'].isnull()]['model'].unique())
#print(df['size'].unique())

In [664]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [665]:
df.shape

(426880, 18)

In [666]:
df.tail()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
426875,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,wy
426876,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,wy
426877,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,wy
426878,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,wy
426879,7301591129,wyoming,30590,2019.0,bmw,4 series 430i gran coupe,good,NaN,gas,22716.0,clean,other,WBA4J1C58KBM14708,rwd,NaN,coupe,NaN,wy


In [667]:
df.isnull().sum()

id                   0
region               0
price                0
year              1205
manufacturer     17646
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
VIN             161042
drive           130567
size            306361
type             92858
paint_color     130203
state                0
dtype: int64

In [668]:
#dropping columns 'VIN' and 'size', those features are not important to predict price, Size should be covered by model, also both have significant number of missing data
df = df.drop(columns=['VIN','size'])
df.shape

(426880, 16)

In [669]:
# Create a Series that maps models to manufacturers (where manufacturer is not null)

model_to_manufacturer = df[df.manufacturer.notnull()].drop_duplicates(subset='model').set_index('model')['manufacturer']
#print(model_to_manufacturer)

# Fill missing manufacturers based on the model_to_manufacturer mapping
df.loc[df.manufacturer.isnull(), 'manufacturer'] = df['model'].map(model_to_manufacturer)
if DEBUG:
    print(df.isnull().sum())


id                   0
region               0
price                0
year              1205
manufacturer     16778
model             5277
condition       174104
cylinders       177678
fuel              3013
odometer          4400
title_status      8242
transmission      2556
drive           130567
type             92858
paint_color     130203
state                0
dtype: int64


In [670]:
#dropping null rows for 'year', 'model' and 'manufacturer' as null rows are comparatively low
df = df.dropna(subset=['year', 'manufacturer', 'model'])
if DEBUG:
    print(df.isnull().sum())

id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       164577
cylinders       167254
fuel              2553
odometer          4138
title_status      7718
transmission      2271
drive           121267
type             84438
paint_color     121534
state                0
dtype: int64


In [671]:
# Extract numeric part from the 'cylinders' column
df['cylinders'] = df['cylinders'].str.extract('(\d+)').astype(float)

median_cylinders =  df.groupby('model')['cylinders'].median()
overall_median = df['cylinders'].median()  # Calculate overall median for any missing model medians
print(overall_median)

df['cylinders'] = df.apply(
    lambda row: median_cylinders[row['model']] if pd.isnull(row['cylinders']) else row['cylinders'],
    axis=1
)
df['cylinders'] = df.apply(
   lambda row: overall_median if pd.isnull(row['cylinders']) else row['cylinders'],
    axis=1
)
print(df.cylinders.unique())
if DEBUG:
    print(df.isnull().sum())


6.0
[ 8.   6.   4.   5.   4.5  9.  10.   3.   7.  12.   5.5  6.5]
id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       164577
cylinders            0
fuel              2553
odometer          4138
title_status      7718
transmission      2271
drive           121267
type             84438
paint_color     121534
state                0
dtype: int64


In [672]:
def fill_odometer(row):
    if pd.isnull(row['odometer']):
     # Get the odometer value for the year of the current row

        year_odometer = yearly_odometer.get(row['year'])

        if pd.isnull(year_odometer):
            # Find the closest year with available data

            closet_year = yearly_odometer.dropna().sub(row['year']).abs().idxmin()
            return yearly_odometer[closet_year]
        else:
            return year_odometer
    else:
        return row['odometer']


In [673]:
# for odometer assumption, use median value by year and use that to estimate missing values.
yearly_odometer = df.groupby(['year'])['odometer'].median()

df['odometer'] = df.apply(fill_odometer, axis=1)
#plt.plot(df['year'], df['odometer'])
if DEBUG:
    print(df.isnull().sum())

id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition       164577
cylinders            0
fuel              2553
odometer             0
title_status      7718
transmission      2271
drive           121267
type             84438
paint_color     121534
state                0
dtype: int64


In [674]:
#print(df.condition.value_counts(dropna=False, ascending=False, normalize=True))
#as 28% of the rows are in 'good' condition, 2nd highest after NAN, and very very few in fair, new and salvage, it would be fair to assume that NAN will be good. 

df['condition'] = df['condition'].fillna('Not Specified')
if DEBUG:
    #print(df.condition.value_counts(dropna=False, ascending=False, normalize=True))
    print(df.isnull().sum())




id                   0
region               0
price                0
year                 0
manufacturer         0
model                0
condition            0
cylinders            0
fuel              2553
odometer             0
title_status      7718
transmission      2271
drive           121267
type             84438
paint_color     121534
state                0
dtype: int64


In [675]:
#filling all other Null as "Not Specified"

df = df.fillna('Not Specified')
if DEBUG:
    #print(df.condition.value_counts(dropna=False, ascending=False, normalize=True))
    print(df.isnull().sum())

id              0
region          0
price           0
year            0
manufacturer    0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
type            0
paint_color     0
state           0
dtype: int64


In [676]:
print(df.fuel.value_counts(dropna=False, ascending=False, normalize=True))


fuel
gas              0.842961
other            0.070949
diesel           0.063201
hybrid           0.012596
Not Specified    0.006308
electric         0.003985
Name: proportion, dtype: float64


### Data Preparation

After our initial exploration and fine tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight on drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine tuning their inventory.